# AIM 5001 Week 10 Assignment
# Tidying and Transforming Data

### 1. Create the data
* The example .csv file has been created and uploaded to GitHub. The .csv file will be used in the following questions.
* Applicable python libraries are imported below.

In [1]:
import pandas as pd
import numpy as np

### 2. Read the data and transform it to "long" format
Import the .csv file and take the city names as header, the airlines and on time or delay columns as multi-index. Show the table first.

In [2]:
# set first two columns as index and first row as column names, the thousand indicator comma will be set invisable.
original=pd.read_csv(r'https://raw.githubusercontent.com/chenkecoco1/AIM-5001/master/AMI5001_WEEK10.csv',index_col=[0,1],header=0,thousands=',')
# try to see the table
original

Los Angeles  Phoenix  San Diego  San Francisco  Seattle
ALASKA  on time        497.0    221.0      212.0          503.0   1841.0
NaN     delayed         62.0     12.0       20.0          102.0    305.0
        NaN              NaN      NaN        NaN            NaN      NaN
AM WEST on time        694.0   4840.0      383.0          320.0    201.0
NaN     delayed        117.0    415.0       65.0          129.0     61.0

According to the original table, null values are mixed with valid values. The null values are also mixed with row names. The next step will be dropping the null values and setting valid row and column names.

In [3]:
# drop the null value row and reset the index for further fix, store the result as fillin
# drop the null values and reset index
fillin=original.dropna().reset_index()

# after resetting index, two new columns appears, set the column names for them
fillin.columns=['airlines','if_delayed','Los Angeles','Phoenix','San Diego','San Francisco','Seattle']
# fill in the null value of airlines
fillin['airlines'].fillna(method='ffill',inplace=True)
# remove the decimal
for column in ['Los Angeles','Phoenix','San Diego','San Francisco','Seattle']:
    fillin[column]=fillin[column].astype(int)
# show the new table
fillin

,airlines,if_delayed,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
0,ALASKA,on time,497,221,212,503,1841
1,ALASKA,delayed,62,12,20,102,305
2,AM WEST,on time,694,4840,383,320,201
3,AM WEST,delayed,117,415,65,129,61


Now the method of reshaping data will be discussed below. According to the fillin table, the airlines and cities can be taken as 2 dimensions, which can be taken as the key values, while the numbers of flights (on time or delayed) can be taken as the measure. Therefore, the numbers of flights of each airline and city can be taken as a single observation. Total four columns should be contained in the long format table, with headings of 'cities', 'airlines', 'on time', and 'delayed', as the numbers of flights are divided into on time and delayed.

In [4]:
# set the key values as index again and transpose the table as pre_tidy
pre_tidy=fillin.set_index(['airlines','if_delayed']).T

# use .stack() to convert it to the long format as tidy
tidy=pre_tidy.stack(level='airlines').reset_index()
# name the column headings
tidy.columns=['city','airlines','delayed','on time']
# show the long format table
tidy

,city,airlines,delayed,on time
0,Los Angeles,ALASKA,62,497
1,Los Angeles,AM WEST,117,694
2,Phoenix,ALASKA,12,221
3,Phoenix,AM WEST,415,4840
4,San Diego,ALASKA,20,212
5,San Diego,AM WEST,65,383
6,San Francisco,ALASKA,102,503
7,San Francisco,AM WEST,129,320
8,Seattle,ALASKA,305,1841
9,Seattle,AM WEST,61,201


### 3. Questions
Q1. For each city, which airline had the best on time performance?
* Create a new_tidy table to calculate the on time rate of each city of each airline.

In [5]:
# create a new_tidy table for on time rate comparison
new_tidy=tidy.copy()
# iterate each row to add a new column showing the on time rate of each city of each airline
new_tidy['on time rate (%)']= new_tidy.apply(lambda new_tidy: round(new_tidy['on time']/(new_tidy['on time']+ new_tidy['delayed']),2), axis=1)
# see the new added column
new_tidy

,city,airlines,delayed,on time,on time rate (%)
0,Los Angeles,ALASKA,62,497,0.89
1,Los Angeles,AM WEST,117,694,0.86
2,Phoenix,ALASKA,12,221,0.95
3,Phoenix,AM WEST,415,4840,0.92
4,San Diego,ALASKA,20,212,0.91
5,San Diego,AM WEST,65,383,0.85
6,San Francisco,ALASKA,102,503,0.83
7,San Francisco,AM WEST,129,320,0.71
8,Seattle,ALASKA,305,1841,0.86
9,Seattle,AM WEST,61,201,0.77


In order to show the final result, the sorted table will be shown below in a descending order by each city and on time rate. The first record of each city will be the one with higher on time rate.  

In [6]:
# sort the table by city and on time rate
q1=new_tidy.sort_values(by=['city','on time rate (%)'],ascending=False).reset_index()
# show the rows with odd new index number
q1.loc[0::2,:]

,index,city,airlines,delayed,on time,on time rate (%)
0,8,Seattle,ALASKA,305,1841,0.86
2,6,San Francisco,ALASKA,102,503,0.83
4,4,San Diego,ALASKA,20,212,0.91
6,2,Phoenix,ALASKA,12,221,0.95
8,0,Los Angeles,ALASKA,62,497,0.89


According to the on time rate result, all cities shows Alaska Airlines (ALASKA) had a higher on time rate, which means Alaska  Airlines had the best on time performance for all the five cities.

Q2. Which airline had the best overall on time performance?
* Group by each airline to calculate the total flights and on time flights, and calculate the on time rate.

In [7]:
# sum the total delayed flights of each airline
sum_delayed=tidy.groupby('airlines')['delayed'].sum()
# sum the total on time flights of each airline
sum_ontime=tidy.groupby('airlines')['on time'].sum()
# calculate the overall on time rate of each airline
round(sum_ontime/(sum_delayed+sum_ontime),2)

airlines
ALASKA     0.87
AM WEST    0.89
dtype: float64

According to the result, America West Airline (AM WEST) had a higher overall on time rate, which indicates it had the best overall on time performance.

Q3. In which city did flights have the best overall on time performance?

In [8]:
# sum the total delayed flights of each city
sum_delayed_city=tidy.groupby('city')['delayed'].sum()
# sum the total on time flights of each city
sum_ontime_city=tidy.groupby('city')['on time'].sum()
# calculate the overall on time rate of each city
round(sum_ontime_city/(sum_delayed_city+sum_ontime_city),2)

city
Los Angeles      0.87
Phoenix          0.92
San Diego        0.88
San Francisco    0.78
Seattle          0.85
dtype: float64

According to the result above, flights had a highest overall flights on time rate in Phoenix, which indicates in Phoenix the flights had the best on time performance.

### 4. Transform it back to "wide" format
To transform the data to "wide" format, the airlines and cities need to be separated in row index and column names. The original table includes spaces which are meaningless null values. The new "wide" format tables need to get rid of the meaningless units. I recommend taking the airlines as the column names and cities as index. It is easier to read the delayed and on time flights numbers vertically as it is more understandable to keep the index unified. 

In [9]:
# pivot the tidy table to wide format, city as index and airlines as column names
pivottidy=tidy.pivot(index='city', columns='airlines', values=['delayed','on time'])
# show the example table
pivottidy

delayed         on time        
airlines       ALASKA AM WEST  ALASKA AM WEST
city                                         
Los Angeles        62     117     497     694
Phoenix            12     415     221    4840
San Diego          20      65     212     383
San Francisco     102     129     503     320
Seattle           305      61    1841     201